# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [4]:
spark = SparkSession.builder.appName('CustomerChurn').getOrCreate()

In [5]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,header=True)

In [6]:
final_data = data.select('Age','Total_Purchase','Years','Num_Sites',year('Onboard_date').alias('Onboard_Year'),'Churn')

In [7]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Years','Onboard_Year','Num_Sites'],outputCol='features')

In [8]:
log_reg_customer = LogisticRegression(featuresCol='features',labelCol='Churn')

In [9]:
pipeline = Pipeline(stages=[assembler,log_reg_customer])

In [10]:
train_customer_data,test_customer_data = final_data.randomSplit([0.8,0.2])

In [11]:
fit_model = pipeline.fit(train_customer_data)

In [12]:
results = fit_model.transform(test_customer_data)

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [14]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [16]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [17]:
AUC = my_eval.evaluate(results)

In [18]:
AUC

In [19]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,header=True)

In [20]:
new_customers_final = new_customers.select('Age','Company','Total_Purchase','Years','Num_Sites',year('Onboard_date').alias('Onboard_Year'))

In [21]:
new_customer_churns = fit_model.transform(new_customers_final)

In [22]:
new_customer_churns.select('Company','prediction').show()